In [1]:
from database.db_config import current_db as db

In [2]:
#  Alter the headers types, before querying.
db.execute_ddl('ALTER TABLE "All continents updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer, ALTER COLUMN "Date" SET DATA TYPE date USING to_date(cast("Date" as TEXT), \'yyyy-mm-dd\')')
db.execute_ddl('ALTER TABLE "All countries updated" ALTER COLUMN "TotalCases" TYPE INT USING "TotalCases"::integer,ALTER COLUMN "NewCases" TYPE INT, ALTER COLUMN "TotalDeaths" TYPE INT USING "TotalDeaths"::integer,ALTER COLUMN "NewDeaths" TYPE INT, ALTER COLUMN "TotalRecovered" TYPE INT USING "TotalRecovered"::integer,ALTER COLUMN "NewRecovered" TYPE INT, ALTER COLUMN "ActiveCases" TYPE INT USING "ActiveCases"::integer,ALTER COLUMN "SeriousCritical" TYPE INT USING "SeriousCritical"::integer,ALTER COLUMN "Date" SET DATA TYPE date USING to_date(cast("Date" as TEXT), \'yyyy-mm-dd\')')


DDL executed successfully.
DDL executed successfully.


In [3]:
countries_minMax = db.sql_query('SELECT "Country", "TotalCases" FROM "All countries updated"'
                        'WHERE "TotalCases" IN  ((SELECT MIN("TotalCases")FROM "All countries updated" LIMIT 1),'
						                       '((SELECT MAX("TotalCases") FROM "All countries updated"LIMIT 1)))')
print(countries_minMax)

+---------+------------+
| Country | TotalCases |
+---------+------------+
|   USA   |  19670613  |
| Vanuatu |     1      |
+---------+------------+


In [4]:
# TotalCases Min and Max by a continent.
continents_minMax = db.sql_query('SELECT "Continent", "TotalCases" FROM "All continents updated"'
                        'WHERE "TotalCases" IN  ((SELECT MIN("TotalCases")FROM "All continents updated" WHERE "Continent" <>  \'World\' LIMIT 1),'
						                       '((SELECT MAX("TotalCases") FROM "All continents updated" WHERE "Continent" <>  \'World\'LIMIT 1)))')
print(continents_minMax)

+-----------+------------+
| Continent | TotalCases |
+-----------+------------+
|   Europe  |  22864797  |
|  Oceania  |   47923    |
+-----------+------------+


In [5]:
# Top 3 countries with the highest number of cases by Continents.
top3_totalCases = db.sql_query('SELECT * '
                                'FROM (SELECT "Country","Continent","TotalCases",RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "TotalCases_Rank","ActiveCases","SeriousCritical"'
                                'FROM (SELECT "Country_id","Continent_id"'
                                        'FROM "All Countries") AS t1 JOIN  (SELECT * '
                                                                            'FROM "All Continents") AS t2 USING("Continent_id")'
                                        'JOIN  (SELECT "Country","Country_id","TotalCases","ActiveCases","SeriousCritical"'
                                                'FROM "All countries updated" ) AS t3 USING("Country_id")) AS "t1"'
                                'WHERE "TotalCases_Rank" <=3 ')

print(top3_totalCases)

+------------------+---------------+------------+-----------------+-------------+-----------------+
|     Country      |   Continent   | TotalCases | TotalCases_Rank | ActiveCases | SeriousCritical |
+------------------+---------------+------------+-----------------+-------------+-----------------+
|   South Africa   |     Africa    |  1011871   |        1        |    134826   |       546       |
|     Morocco      |     Africa    |   433029   |        2        |    26182    |       769       |
|     Tunisia      |     Africa    |   133204   |        3        |    27441    |       332       |
|      India       |      Asia     |  10224797  |        1        |    269840   |       8944      |
|      Turkey      |      Asia     |  2162775   |        2        |    105207   |       4251      |
|       Iran       |      Asia     |  1206373   |        3        |    190808   |       5174      |
|      Russia      |     Europe    |  3078035   |        1        |    551461   |       2300      |


In [6]:
# Countries that are located in Oceania continent.
oceania_countries = db.sql_query('SELECT "Country" '
                                 'FROM "All Countries" '
                                 'WHERE "Continent_id" IN (SELECT "Continent_id" '
                                                            'FROM "All Continents" '
                                 '                           WHERE "Continent" = \'Oceania\')')
print(oceania_countries)

+------------------+
|     Country      |
+------------------+
|    Australia     |
|   New Zealand    |
| French Polynesia |
| Papua New Guinea |
|       Fiji       |
|  New Caledonia   |
+------------------+


In [7]:
# The count of countries in each continent.
countries_perContinent = db.sql_query('SELECT "Continent","Countires_count" FROM (SELECT * FROM "All Continents") AS t1 '
                                      'JOIN (SELECT "Continent_id", COUNT("Country") AS "Countires_count" '
                                      'FROM "All Countries" G'
                                      'ROUP BY "Continent_id") AS t2 '
                                      'USING("Continent_id") '
                                      'ORDER BY "Countires_count" DESC')
print(countries_perContinent)

+---------------+-----------------+
|   Continent   | Countires_count |
+---------------+-----------------+
|     Africa    |        57       |
|      Asia     |        49       |
|     Europe    |        48       |
| North America |        39       |
| South America |        14       |
|    Oceania    |        6        |
+---------------+-----------------+


In [8]:
# The country with the highest "TotalCases" in each continent.
country_high_perContinent = db.sql_query('SELECT "Country","Continent","TotalCases" '
                                         'FROM (SELECT "Country","Continent","TotalCases" ,RANK() OVER(PARTITION BY "Continent" ORDER BY "TotalCases" DESC) AS "rank" '
                                                'FROM (SELECT "Country","Country_id","TotalCases" '
                                                       'FROM "All countries updated" ) AS t1  '
                                                       'JOIN (SELECT "Country_id", "Continent_id" FROM "All Countries") AS t2 '
                                                        'USING("Country_id") '
                                                        'JOIN (SELECT "Continent_id","Continent" FROM "All Continents") AS t3 '
                                                        'USING("Continent_id")) AS t4 '
                                         'WHERE "rank" =1;')
print(country_high_perContinent)

+--------------+---------------+------------+
|   Country    |   Continent   | TotalCases |
+--------------+---------------+------------+
| South Africa |     Africa    |  1011871   |
|    India     |      Asia     |  10224797  |
|    Russia    |     Europe    |  3078035   |
|     USA      | North America |  19670613  |
|  Australia   |    Oceania    |   28337    |
|    Brazil    | South America |  7486094   |
+--------------+---------------+------------+


In [9]:
# Top 5 countries with active cases.
top5_countries_active = db.sql_query('SELECT "Country","ActiveCases" '
                                     'FROM "All countries updated" '
                                     'WHERE "ActiveCases" IS NOT NULL '
                                     'ORDER BY "ActiveCases" DESC LIMIT 5;')
print(top5_countries_active)

+---------+-------------+
| Country | ActiveCases |
+---------+-------------+
|   USA   |   7786087   |
|  France |   2308815   |
|  Brazil |    779517   |
| Belgium |    575804   |
|  Italy  |    575221   |
+---------+-------------+


In [10]:
# The date that israel had the highest value of new cases.
israel_newCases_date = db.sql_query('SELECT "Date" AS "max_NewCases_date" '
                                    'FROM "Israel" '
                                    'WHERE "NewCases" = (Select MAX("NewCases") '
                                    '                    FROM "Israel")')
print(israel_newCases_date)

+-------------------+
| max_NewCases_date |
+-------------------+
|     2020-09-27    |
+-------------------+


In [11]:
# Ranking each continent by its active cases.
continent_active_rank = db.sql_query('SELECT "Continent","ActiveCases", RANK() OVER(ORDER BY "ActiveCases" DESC) '
                                     'FROM "All continents updated" '
                                     'WHERE "Continent" <> \'World\'')
print(continent_active_rank)

+---------------+-------------+------+
|   Continent   | ActiveCases | rank |
+---------------+-------------+------+
|     Europe    |   11163359  |  1   |
| North America |   8291525   |  2   |
|      Asia     |   1220972   |  3   |
| South America |   1111665   |  4   |
|     Africa    |    379996   |  5   |
|    Oceania    |    13471    |  6   |
+---------------+-------------+------+


In [12]:
israel_minMax_dates = db.sql_query('SELECT MIN("Date") as "Earliest Date",MAX("Date") as "Latest Date" '
                                   'FROM "Israel"')
print(israel_minMax_dates)

+---------------+-------------+
| Earliest Date | Latest Date |
+---------------+-------------+
|   2020-08-02  |  2020-12-28 |
+---------------+-------------+
